In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Session and Events


### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.9 MB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [13]:
# @title Authentication to access to GCP
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.

In [4]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Session store

In [5]:
import sqlite3
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService, Session

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/

if True: # InMemory
  session_service = InMemorySessionService()

else: # SQLite3
  db_url = "sqlite:///./mydatabase.db"
  session_service = DatabaseSessionService(db_url=db_url)

In [16]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.

"""

# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
generate_content_config= types.GenerateContentConfig(
  temperature=0.2,
  max_output_tokens=8192,
)

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="Search_assistant",
      model=MODEL_NAME,
      instruction=PROMPT,
      generate_content_config = generate_content_config,
      description="An assistant that can search the web.",
      tools=[google_search] # Use Google search tool
)

In [7]:
from google.adk.sessions import InMemorySessionService, Session

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME,
                                         user_id=USER_ID,
                                         state={"initial_key": "initial_value"})

In [15]:
from google.adk.runners import Runner

# query = "What is the exchange rate from US dollars to Korean currency recently ?"
query = "오늘 원달러 환율은 얼마인가요 ?"

# Runner : An in-memory Runner for testing and development.
# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner
runner = Runner(agent=agent,
                app_name=session.app_name,
                session_service=session_service)

# https://googleapis.github.io/python-genai/genai.html#genai.types.Content
content = types.Content(role='user', parts=[types.Part(text=query)])

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner.run
events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:

  print(event)

  if event.is_final_response():
      final_response = event.content.parts[0].text
      display(Markdown(final_response))

      grounding_metadata = event.grounding_metadata
      print("\n\ngrounding_metadata: \n\n", grounding_metadata)

  else:
      print("No results\n")

content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='오늘 원/달러 환율은 여러 출처에서 조금씩 다르게 나타나지만, 대략 1달러당 1429원에서 1438원 사이에서 거래되고 있습니다.\n\n몇 가지 출처의 환율 정보는 다음과 같습니다 (2025년 4월 30일 기준):\n\n*   씨티은행: 1달러당 1429.15원 (오전 9시 57분 기준) [6]\n*   알파스퀘어: 1달러당 1432.24원 [13]\n*   서울외국환중개: 1달러당 1438.50원 [7]\n*   다음 금융 (하나은행 고시 기준): 1달러당 1438.80원 (4월 25일 22:06 기준) [5]\n*   XE.com: 1달러당 1431.7원 또는 1438.26원 [2, 4]\n*   Wise: 1달')], role='model') grounding_metadata=GroundingMetadata(grounding_chunks=[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='daum.net', title='daum.net', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALImyrbIm7zPszF0G8qG2-oheg6wNEZ7S_KhCB3zvf7DCYS2okqk3H4-R9DT6H4O5ySrrWfml4O-foKXia0ByTBGnq-ajXfROZapyIr9SWLY-KBflNR0x3_mDCxxLM='))], grounding_supports=[GroundingSupport(confidence_scores=[0.8303966], grounding

오늘 원/달러 환율은 여러 출처에서 조금씩 다르게 나타나지만, 대략 1달러당 1429원에서 1438원 사이에서 거래되고 있습니다.

몇 가지 출처의 환율 정보는 다음과 같습니다 (2025년 4월 30일 기준):

*   씨티은행: 1달러당 1429.15원 (오전 9시 57분 기준) [6]
*   알파스퀘어: 1달러당 1432.24원 [13]
*   서울외국환중개: 1달러당 1438.50원 [7]
*   다음 금융 (하나은행 고시 기준): 1달러당 1438.80원 (4월 25일 22:06 기준) [5]
*   XE.com: 1달러당 1431.7원 또는 1438.26원 [2, 4]
*   Wise: 1달



grounding_metadata: 

 grounding_chunks=[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='daum.net', title='daum.net', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALImyrbIm7zPszF0G8qG2-oheg6wNEZ7S_KhCB3zvf7DCYS2okqk3H4-R9DT6H4O5ySrrWfml4O-foKXia0ByTBGnq-ajXfROZapyIr9SWLY-KBflNR0x3_mDCxxLM='))] grounding_supports=[GroundingSupport(confidence_scores=[0.8303966], grounding_chunk_indices=[0], segment=Segment(end_index=528, part_index=None, start_index=429, text='*   다음 금융 (하나은행 고시 기준): 1달러당 1438.80원 (4월 25일 22:06 기준) [5]'))] retrieval_metadata=RetrievalMetadata(google_search_dynamic_retrieval_score=None) retrieval_queries=None search_entry_point=SearchEntryPoint(rendered_content='<style>\n.container {\n  align-items: center;\n  border-radius: 8px;\n  display: flex;\n  font-family: Google Sans, Roboto, sans-serif;\n  font-size: 14px;\n  line-height: 20px;\n  padding: 8px 12px;\n}\n.chip {\n  display: inline-block;\n  border: solid 1px;\n  b

In [17]:
session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

ListSessionsResponse(sessions=[Session(id='dbea35f0-4339-47a8-91f9-e68da49973d7', app_name='Search_Assistant', user_id='shins', state={}, events=[], last_update_time=1745976803.725602)])

In [18]:
from pprint import pprint

pprint(session_service.sessions[session.app_name][session.user_id][session.id].events)


[Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='What is the situation of the world economy?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-b61cee0c-628d-44a1-8d98-431ed5d80ad3', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='F9t25Jx1', timestamp=1745976064.452232),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="The global economic situation is characterized by steady but slow growth, moderating infl

In [12]:
from google.adk.sessions import InMemorySessionService, Session

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {session.id}")
print(f"Application Name (`app_name`): {session.app_name}")
print(f"User ID (`user_id`):         {session.user_id}")
print(f"State (`state`):           {session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {session.last_update_time:.2f}")
print(f"---------------------------------")


--- Examining Session Properties ---
ID (`id`):                dbea35f0-4339-47a8-91f9-e68da49973d7
Application Name (`app_name`): Search_Assistant
User ID (`user_id`):         shins
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1745976013.37
---------------------------------


In [ ]:
# Clean up (optional for this example)
session_service.delete_session(app_name=session.app_name,
                               user_id=session.user_id,
                               session_id=session.id)

## End of notebook